In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch 
from torch import nn

import os
import pandas as pd
from torch.utils.data import Dataset, Subset

import h5py
import numpy as np

In [ ]:
arr = np.random.randn(1000)

with h5py.File('random.hdf5', 'w') as f:
    dset = f.create_dataset("default", data=arr)

In [ ]:
gamma_f= h5py.File("/content/drive/MyDrive/year planning/gamma.hdf5", 'r') 

In [ ]:
energy = gamma_f['energy'][:]
layer_0 = gamma_f['layer_0'][:]
layer_1 = gamma_f['layer_1'][:]
layer_2 = gamma_f['layer_2'][:]
overflow = gamma_f['overflow'][:]

In [ ]:
eplus_f = h5py.File("/content/drive/MyDrive/year planning/eplus.hdf5", 'r') 

In [ ]:
energy_ep = eplus_f['energy'][:]
layer_0_ep = eplus_f['layer_0'][:]
layer_1_ep = eplus_f['layer_1'][:]
layer_2_ep = eplus_f['layer_2'][:]
overflow_ep = eplus_f['overflow'][:]

In [ ]:
class network(nn.Module):
    def __init__(self,):
        super().__init__()       
      
        self.Conv1_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, 
                      kernel_size=(1, 16)
                      ), 
            nn.ReLU(), 
            nn.BatchNorm2d(64)
        )
        self.Conv1_2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=32, 
                      kernel_size=(1, 8)
                      ), 
            nn.ReLU(), 
            nn.BatchNorm2d(32),
            nn.MaxPool2d((1, 2))
        )
        self.Conv1_3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=16, 
                      kernel_size=(1, 8)
                      ), 
            nn.ReLU(), 
            nn.BatchNorm2d(16),
        )
        self.Conv1_4 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16,
                      kernel_size=(3, 26)
                      ), 
            nn.ReLU(),
        ) 

        self.Conv2_1 = nn.Sequential(
            nn.Conv2d(1, 64, (3, 3)), 
            nn.ReLU(), 
            nn.BatchNorm2d(64)
        )
        self.Conv2_2 = nn.Sequential(
            nn.Conv2d(64, 32, (5, 5)), 
            nn.ReLU(), 
            nn.BatchNorm2d(32),
            #nn.MaxPool2d((2, 2))
        )
        self.Conv2_3 = nn.Sequential(
            nn.Conv2d(32, 16, (1, 1)), 
            nn.ReLU(), 
            nn.BatchNorm2d(16),
            #nn.MaxPool2d((2, 2))
        )
        self.Conv2_4 = nn.Sequential(
            nn.Conv2d(16, 16, (6, 2)), 
            nn.ReLU(),
        )

        self.Conv3_1 = nn.Sequential(
            nn.Conv2d(1, 64, (3, 1)), 
            nn.ReLU(), 
            nn.BatchNorm2d(64)
        )
        self.Conv3_2 = nn.Sequential(
            nn.Conv2d(64, 32, (5, 1)), 
            nn.ReLU(), 
            nn.BatchNorm2d(32),
            
        )
        self.Conv3_3 = nn.Sequential(
            nn.Conv2d(32, 16, (3, 2)), 
            nn.ReLU(), 
            nn.BatchNorm2d(16),
           
        )
        self.Conv3_4 = nn.Sequential(
            nn.Conv2d(16, 16, (4, 1)), 
            nn.ReLU(),
        )

        self.input1 = nn.Linear(16, 16)
        
        self.input2 = nn.Linear(12*12,20)
        self.input3 = nn.Linear(12*6,10)
        
        self.bn1 = nn.BatchNorm1d(30)
        self.bn2 = nn.BatchNorm1d(20)
        self.bn3 = nn.BatchNorm1d(10)
        
        self.drop1 = nn.Dropout(p=0.2)
        self.drop2 = nn.Dropout(p=0.2)
        self.drop3 = nn.Dropout(p=0.2)

        self.lin = nn.Linear(60,1)
        
    def forward(self,x,y,z):
        x = x.reshape([16,1,3,96]).float()
        x = self.Conv1_1(x)
        x = self.Conv1_2(x)
        x = self.Conv1_3(x)
        x = self.Conv1_4(x)
        y = y.reshape([16,1,12,12]).float()
        y = self.Conv2_4(self.Conv2_3(self.Conv2_2(self.Conv2_1(y))))
        z = z.reshape([16,1,12,6]).float()
        z = self.Conv3_1(z)
        z = self.Conv3_2(z)
        z = self.Conv3_3(z)
        z = self.Conv3_4(z)

        print(x.shape, y.shape, z.shape)
        #z = self.Conv3_4(self.Conv3_3(self.Conv3_2(self.Conv3_1(z))))
        x = self.input1(x.flatten(start_dim=1))
        y = self.input2(y.flatten(start_dim=1))
        z = self.input3(z.flatten(start_dim=1))
        
        h = torch.cat((self.drop1(self.bn1(x)), self.drop2(self.bn2(y)), self.drop3(self.bn3(z))),axis=1)
        
        h = self.lin(h)
        return h

In [ ]:

def conv_block(input_size, output_size):
    block = nn.Sequential(
        nn.Conv2d(input_size, output_size, (3, 3)), nn.ReLU(), nn.BatchNorm2d(output_size), nn.MaxPool2d((2, 2)),
    )

    return block

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, labels, layer_0, layer_1,layer_02):
        self.labels = labels
        self.layer_0 = layer_0 
        self.layer_1 = layer_1
        self.layer_2 = layer_2
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image1 = layer_0[idx]
        image2 = layer_1[idx]
        image3 = layer_2[idx]
        label = self.labels[idx]
        return (image1,image2,image3), label

In [ ]:
dataset_gamma = CustomImageDataset(np.ones(100000),layer_0,layer_1,layer_2)
dataset_ep = CustomImageDataset(np.zeros(100000),layer_0_ep,layer_1_ep,layer_2_ep)
total_data = torch.utils.data.ConcatDataset([dataset_gamma, dataset_ep])

In [ ]:
N = len(total_data)

# generate & shuffle indices
indices = np.arange(N)
indices = np.random.permutation(indices)
# there are many ways to do the above two operation. (Example, using np.random.choice can be used here too

# select train/test/val, for demo I am using 70,15,15
train_indices = indices [:int(0.7*N)]
val_indices = indices[int(0.7*N):int(0.85*N)]
test_indices = indices[int(0.85*N):]

train_dataset = Subset(total_data, train_indices)
val_dataset = Subset(total_data, val_indices)
test_dataset = Subset(total_data, test_indices)

In [ ]:
len(total_data[0][0])

3

In [ ]:
len(total_data[0])

2

In [ ]:
layer_0.shape

(100000, 3, 96)

In [ ]:
layer_1.shape

(100000, 12, 12)

In [ ]:
layer_2.shape

(100000, 12, 6)

In [ ]:
overflow.shape

(100000, 3)

In [ ]:
heh = network()

In [ ]:
layer_0[0].shape

(3, 96)

In [ ]:
torch.Tensor(layer_0[0]).flatten().shape

torch.Size([288])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
dataloaders = {'train':DataLoader(train_dataset, batch_size=16),'val':DataLoader(val_dataset, batch_size=16), 'test':DataLoader(test_dataset, batch_size=16)}

In [ ]:
loss_fn = loss = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = torch.optim.Adam(heh.parameters(), lr=0.00001)

In [ ]:
dataloaders['train']

In [ ]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, (data,labels) in enumerate(dataloaders['train']):
        # Every data instance is an input + label pair
        data0,data1,data2 = data
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = heh(data0.float(),data1.float(),data2.float())

        # Compute the loss and its gradients
        loss = loss_fn(outputs.float().squeeze(), labels.float())
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(dataloaders['train']) + i + 1
            running_loss = 0.

    return last_loss

In [ ]:
train_loss = []
val_loss = []

In [ ]:
EPOCHS = 30
epoch_number = 0

best_vloss = 1_00000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    heh.train(True)
    avg_loss = train_one_epoch(epoch_number)
    train_loss.append(avg_loss)
    # We don't need gradients on to do reporting
    heh.train(False)

    running_vloss = 0.0
    for i, (vdata,vlabels) in enumerate(dataloaders['val']):
        vdata0,vdata1,vdata2 = vdata
        voutputs = heh(vdata0.float(),vdata1.float(),vdata2.float())
        vloss = loss_fn(voutputs.squeeze(), vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    val_loss.append(avg_vloss)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation


    # Track best performance, and save the model's state
#     if avg_vloss < best_vloss:
#         best_vloss = avg_vloss
#         model_path = 'model_{}_{}'.format(timestamp, epoch_number)
#         torch.save(heh.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
torch.Size([16, 16, 1, 5]) torch.Size([16, 16, 1, 5]) torch.Size([16, 16, 1, 5])


RuntimeError: ignored

In [ ]:
predicts = []
real = []
for i, (vdata,vlabels) in enumerate(dataloaders['test']):
        vdata0,vdata1,vdata2 = vdata
        voutputs = heh(vdata0.float(),vdata1.float(),vdata2.float())
        vloss = loss_fn(voutputs.squeeze(), vlabels)
        pred = voutputs > 0.5
        pred = 1 * pred
        running_vloss += vloss
        for i in pred:
            predicts.append(i.item())
        for i in vlabels:
            real.append(i.item())

from sklearn.metrics import accuracy_score

torch.Size([16, 16, 1, 5]) torch.Size([16, 16, 1, 5]) torch.Size([16, 16, 1, 5])


RuntimeError: ignored

In [ ]:
print(accuracy_score(real,predicts))

NameError: name 'accuracy_score' is not defined

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_loss)
plt.plot([i.item() for i in val_loss])

NameError: name 'train_loss' is not defined

In [ ]:
x.shape

torch.Size([16, 3, 96])

In [ ]:
heh(x.float(),y.float(),z.float())

tensor([[68.4402],
        [30.6635],
        [83.9842],
        [44.9176],
        [ 8.6349],
        [60.6056],
        [26.8921],
        [82.1534],
        [92.0675],
        [ 2.8208],
        [29.8800],
        [90.6870],
        [19.5123],
        [15.8497],
        [86.6526],
        [59.0711]], grad_fn=<AddmmBackward0>)